# Data preparation and ingestion

In [31]:
# Imports
import pandas as pd
import numpy as np
from tqdm.notebook import trange, tqdm
from elasticsearch import Elasticsearch
from statistics import median
from transformers import BigBirdModel, BigBirdTokenizer

## Load the data

In [13]:
# Load in Pandas df
data_path = "../data/ancient_sources.csv.gz"
df = pd.read_csv(data_path, compression="gzip")
df.head()

,author,title,section,text
0,P. Ovidius Naso,The Epistles of Ovid,"Penelope to Ulysses, carte 53",Now corn grows where once Troy stood; and the ...
1,P. Ovidius Naso,Amores,"Liber Primus, ELEGIA 1","ELEGIA 1 Quemadmodum a Cupidine, pro bellis am..."
2,E.C. Marchant,Commentary on Thucydides: Book 7,"book 7, chapter 1, section 1",Ὁ —the art. added because this is a continuati...
3,E.C. Marchant,Commentary on Thucydides: Book 6,"book 6, chapter 1, section 1",ἐβούλοντο —‘the word is here (as in Xen. Hel. ...
4,E.C. Marchant,Commentary on Thucydides: Book 3,"book 3, chapter 1, section 1","θέρους —Thuc. divides the year into θέρος , co..."


In [14]:
# Check size
df.shape

(111752, 4)

In [15]:
# Transform into a list of dicts
df_dict = df.to_dict(orient="records")

In [16]:
# Look at the data
df_dict

[{'author': 'P. Ovidius Naso',
  'title': 'The Epistles of Ovid',
  'section': 'Penelope to Ulysses, carte 53',
  'text': "Now corn grows where once Troy stood; and the ground, fattened by Phrygian blood, produces a rich crop that tempts the hand of the reaper. The half-buried bones of heroes are ploughed up by the crooked share; and rising grass covers the ruins of the houses. Though victorious, you are still absent; nor can I possibly know the cause of your long stay, or in what corner of the world my cruel Ulysses lurks. Whatever stranger touches upon these coasts, is sure to be teased with a thousand questions about you; and, when he departs, is charged with a letter to deliver to you, in whatever region of the world he may chance to see you. We sent to Pylos, the Neleian kingdom of old Nestor; but we thence received no account beside uncertain report. We sent likewise to Sparta; but Sparta, being equally ignorant of the truth, left us uncertain that lands you might be wandering ov

## Generate embeddings for vector search

In [28]:
# Calculate the longest and mean text in our data
counts = []
for src in tqdm(df_dict):
    if type(src["text"]) != str:
        src["text"] = str(src["text"])
        
    counts.append(len(src["text"].split()))

max_words = max(counts)
median_words = median(counts)

print(f"The longest text in our dataset contains {max_words} words, and the median number of words per text is {median_words}")

  0%|          | 0/111752 [00:00<?, ?it/s]

The longest text in our dataset contains 31177 words, and the median number of words per text is 92.0


In [33]:
# Initialize tokenizer and model
model_name = "google/bigbird-roberta-base"
tokenizer = BigBirdTokenizer.from_pretrained(model_name)
model = BigBirdModel.from_pretrained(model_name)

ImportError: 
BigBirdTokenizer requires the SentencePiece library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.
